# Demo: DID example\n\nThis notebook fetches a World Bank indicator (population) and runs a toy DID example across two synthetic groups. Replace with real policy data for a real evaluation.\n

In [ ]:
import pandas as pd\nfrom econ_policy_toolkit.api.worldbank import fetch_indicator\nfrom econ_policy_toolkit.policy.did import diff_in_diff\nimport matplotlib.pyplot as plt\nprint('Fetching population indicator (this is cached)')\ndf = fetch_indicator('SP.POP.TOTL', start=2000, end=2018)\nprint('Sample rows:', len(df))\ndf_small = df[df['country'].isin(['Pakistan','India'])].pivot(index='year', columns='country', values='value').reset_index()\ndf_small = df_small.dropna().head(10)\nprint(df_small.head())\n# Create a toy panel for DID: two groups, treatment = Pakistan after 2008\npanel = []\nfor idx, row in df_small.iterrows():\n    year = row['year']\n    panel.append({'country':'Pakistan','year':year,'y':row['Pakistan'],'treat':1 if year>=2008 else 0})\n    panel.append({'country':'India','year':year,'y':row['India'],'treat':0})\npanel = pd.DataFrame(panel)\nprint(panel.head())\nres = diff_in_diff(panel, outcome='y', treat_col='treat', time_col='year', group_col='country', post_time=2008, covariates=None, cluster_col='country')\nprint(res.summary())\nfig, ax = plt.subplots(figsize=(8,3))\nfor c in panel['country'].unique():\n    s = panel[panel['country']==c].set_index('year')['y']\n    s.plot(ax=ax, label=c)\nax.legend(); ax.set_title('Population (toy)')\nplt.show()\n